# What's Covered?
- A few different options for how to load a datXXX.h5 file with dat_analysis (loading a DatHDF object)
- Some notes/tips about working with DatHDF objects

### Note 
In general use, you will not have to think about most of this as you'll set something like the `get_dat` function below once, and then just use that from then on.

For use on any new computer, there are a couple of things that need to be set up in a `config.toml` to enable the most general use of `dat_analysis`. 

## Import dat_analysis

In [2]:
import dat_analysis

## Loading from explicit filepath
I.e. You want to load a specific `datXXX.h5` file and you know the full absolute or relative path to it.

In [4]:
path_to_experiment_dat = 'experiment_dats/dat6420.h5'
path_to_save_location = 'analyzed_dats/dat6420.h5'  # This is not necessary when the config.toml file is set-up. But a good idea when explicitly loading a single dat like this
dat = dat_analysis.get_dat_from_exp_filepath(path_to_experiment_dat, override_save_path=path_to_save_location, overwrite=False)
dat

Dat6420 - March 3, 2023 14:35:42

Leaving the `dat` object as the last output of a cell, we can see that we have successfully loaded an experiment file and the date and time it was saved.

This dat object is an instance of `DatHDF` which provides easy access information stored in the `.h5` file. See other examples for more information.

In [14]:
type(dat)

dat_analysis.dat.dat_hdf.DatHDF

## Loading from measurement-data
Generally, you'll be loading dats that are saved on the server under the `measurement-data` folder. 

If a path to this folder is set up in the `config.toml` then you can more easily access any dat file.

**Note: I'll leave these examples commented out since you may not already have the config.toml set-up**

The file structure of `measurement-data` is `measurement-data/<host>/<user>/<experiment>/datXXX.h5`. `dat_analysis` assumes it has a path to `measurement-data` already.

In [15]:
host_name = 'qdev-xld'  # The name of the Measurement PC 
user_name = 'Tim'  # The name of the user 
experiment_name = '202211_KondoEntropy'  # Experiment path. Note: this can include more subpaths if necessary e.g. '2023/OhmicTests/Test1/'
# dat = dat_analysis.get_dat(6420, host_name, user_name, experiment_name, raw=False, overwrite=False)

Note: `raw` is related to FastDAC data and allows to easily switch between loading the resampled or raw data that Igor saves when resampling is on.

## Standard Setup

In general, when using this analysis for an experiment, you would define a `get_dat` function in a `.py` file that you can easily import from so that you can easily access the dats for your current experiment from their datnum alone without having to specify everything else every time.

**Note: In this example, I'll specify paths to the example folders here, but if the `config.toml` is set-up, you could use the `dat_analysis.get_dat(...)` function instead as noted**

In [16]:
import dat_analysis

def get_dat(datnum, raw=False, overwrite=False):
    """
    Define a simple get_dat function that knows where to load dats from and save them to

    Note: In a full setup where a path to `measurement-data` and a general save location have already been set in the dat_analysis config.toml, you can use:
        return dat_analysis.get_dat(datnum, host_name, user_name, experiment_name, raw=raw, overwrite=overwrite)
    """
    hdf_name = f"dat{datnum}{'_RAW' if raw else ''}.h5"
    return dat_analysis.get_dat_from_exp_filepath(f'experiment_dats/{hdf_name}', override_save_path=f'analyzed_dats/{hdf_name}', overwrite=overwrite)

In [17]:
dat = get_dat(6420)
dat

Dat6420 - March 3, 2023 14:35:42

For most of the other examples, we'll start with something like this

## Notes on working with DatHDF object

- `dat_analysis` is written such that when you "load" a dat file, you don't actually load much data from the disk until you actually need it. The dat object is mostly an interface for the standard HDF file that `dat_analysis` creates when first loading an experiment dat.
- the HDF that you are actually working with is separate to the experiment `.h5` file, so you don't need to worry about deleting or modifying your experiment data
    - Additionally, if something goes wrong with the datHDF file, you can just load it again with `overwrite=True` and you'll have a clean slate copy from the experiment `.h5` file again.
- The access to the underlying HDF file is (almostly completely) threadsafe and process safe
    - i.e. If you really want to, you should be able to use mutiple threads/processes to access a single dat file (it's not 100% perfect)
    - This is mostly intended for use with `Dash` that ends up doing a lot of muli-threaded/processed operations
- Not everything in the HDF file has a super easy access from the DatHDF object
    - But, the DatHDF object does provide some easy access to the HDF file for reading or writing (see below)
- You can save anything else you like in the HDF file
    - There are some helper functions for saving/loading things to/from HDF in `dat_analysis.hdf_util`

### Interacting with the HDF directly

If you want to interact with the analyzed HDF file directly, you should always access it via the `DatHDF` object to avoid file access issues.

In [18]:
# To read from the HDF:
with dat.hdf_read as f:
    print(f.keys())
    print(f.attrs.keys())

<KeysViewHDF5 ['Data', 'Logs']>
<KeysViewHDF5 ['datnum', 'experiment_data_path', 'new_attr']>


In [19]:
# To write to HDF:
with dat.hdf_write as f:
    f.attrs['new_attr'] = 'test'

with dat.hdf_read as f:
    attr_val = f.attrs['new_attr']
print(attr_val)

test
